In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel("date_temp.xlsx")

# Preprocessing

In the case of the dataset that I have, there is two columns that specify the panels in my data, which are 'assetnum' and 'location' and they are associated to each other, so I need to combine these two columns into one column to use it later as my panel ID 

In [3]:
# Create a new column called 'time_series_id' by combining 'assetnum' and 'location' values
df['time_series_id'] = df['assetnum'] + '_' + df['location']

In [4]:
# Drop columns
columns_to_drop = ['assetnum', 'location', 'Temperature']
df = df.drop(columns=columns_to_drop)

In [5]:
# Define the desired column order
column_order = ['time_series_id', 'month', 'reading']
# Reorder the columns
df = df.reindex(columns=column_order)

Renaming the column to an easier name

In [6]:
df = df.rename(columns={'time_series_id': 'unique_id'})

In [7]:
train = df.loc[df['month'] < '2023-12-01']
test = df.loc[(df['month'] >= '2023-12-01')]
h = test['month'].nunique()

# Model

In the model stage, I am going to search for the best order of ARIMA (p,d,q) for each panel in the dataset, and printout the best oreder alonside the performance metrics, then fit the model for the panel using the best order found by the search, lastly printing out the average of the performance metrics for all of the panels to evaluate the model performance on the whole dataset in general.

In [8]:
# Define a function to evaluate the ARIMA model for each panel
def evaluate_panel_arima_model(panel_data_train, panel_data_test, order):
    model = ARIMA(panel_data_train['reading'], order=order)
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(panel_data_train), end=len(panel_data_train) + len(panel_data_test) - 1)
    mse = mean_squared_error(panel_data_test['reading'], predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(panel_data_test['reading'], predictions)
    mape = mean_absolute_percentage_error(panel_data_test['reading'], predictions)
    return mse, rmse, r2, mape, predictions

In [9]:
# Define the range of parameters to search over
p_values = range(0, 4)  # Example range for p (AR)
d_values = range(0, 4)  # Example range for d (I)
q_values = range(0, 4)  # Example range for q (MA)

In [10]:
# Create a list of unique panel IDs
panel_ids = train['unique_id'].unique()

In [11]:
# Create empty lists to store the best orders and performance metrics for each panel
best_orders = []
mse_scores = []
rmse_scores = []
r2_scores = []
mape_scores = []

# Loop over all possible combinations of parameters and select the best one for each panel
for panel_id in panel_ids:
    panel_data_train = train[train['unique_id'] == panel_id]
    panel_data_test = test[test['unique_id'] == panel_id]

    best_mse = float('inf')
    best_order = None

    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    mse, _, _, _, _ = evaluate_panel_arima_model(panel_data_train, panel_data_test, order)
                    if mse < best_mse:
                        best_mse = mse
                        best_order = order
                except:
                    continue

    # Append the best order to the list
    best_orders.append(best_order)

    # Print the best order
    print("Best Order:", best_order)
    
    # Fit the ARIMA model using the best order for the current panel
    model = ARIMA(panel_data_train['reading'], order=best_order)
    model_fit = model.fit()

    # Generate predictions for the test set of the current panel
    predictions = model_fit.predict(start=len(panel_data_train), end=len(panel_data_train) + len(panel_data_test) - 1)

    # Evaluate the model's performance on the test set of the current panel
    mse = mean_squared_error(panel_data_test['reading'], predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(panel_data_test['reading'], predictions)
    mape = mean_absolute_percentage_error(panel_data_test['reading'], predictions)

    # Append the performance metrics to the respective lists
    mse_scores.append(mse)
    rmse_scores.append(rmse)
    r2_scores.append(r2)
    mape_scores.append(mape)

    # Print the evaluation metrics for the current panel
    print("Evaluation Metrics - Panel ID: {}".format(panel_id))
    print("MSE:", mse)
    print("RMSE:", rmse)
    print("R^2:", r2)
    print("MAPE:", mape)
    print("------------------------")

Best Order: (0, 1, 0)
Evaluation Metrics - Panel ID: AS000302_07-790-GF-0100
MSE: 0.0
RMSE: 0.0
R^2: 1.0
MAPE: 0.0
------------------------
Best Order: (2, 1, 0)
Evaluation Metrics - Panel ID: AS001653_07-916-Z11-SEC03
MSE: 1.442939067735537e+17
RMSE: 379860377.99901384
R^2: -0.00501517303424337
MAPE: 1.3699080533972383
------------------------
Best Order: (2, 2, 2)
Evaluation Metrics - Panel ID: AS001654_07-916-Z17-SEC02
MSE: 56707295871.244446
RMSE: 238132.93739263463
R^2: 0.9652340426032601
MAPE: 0.002456953613427926
------------------------
Best Order: (0, 1, 0)
Evaluation Metrics - Panel ID: AS001655_07-8124-GF
MSE: 34086010467855.332
RMSE: 5838322.573124521
R^2: -0.5181314252793652
MAPE: 0.14948070342151398
------------------------
Best Order: (1, 1, 1)
Evaluation Metrics - Panel ID: AS001656_07-8125-GF
MSE: 118377277076960.55
RMSE: 10880132.217806939
R^2: -59.87986271010644
MAPE: 0.18598947439090355
------------------------
Best Order: (3, 2, 2)
Evaluation Metrics - Panel ID: AS

In [12]:
# Calculate the average performance metrics for all panels
avg_mse = np.mean(mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_r2 = np.mean(r2_scores)
avg_mape = np.mean(mape_scores)

# Print the average performance metrics
print("Average Performance Metrics:")
print("Average MSE:", avg_mse)
print("Average RMSE:", avg_rmse)
print("Average R^2:", avg_r2)
print("Average MAPE:", avg_mape)

Average Performance Metrics:
Average MSE: 2523532685533975.5
Average RMSE: 9677400.778356532
Average R^2: -11.491001975969136
Average MAPE: 0.264328038702531
